In [1]:
import os

In [2]:
def get_folders(location):
    return [name for name in os.listdir(location) if os.path.isdir(os.path.join(location, name))]

states = get_folders('/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/SVI2018 census tracts with death rate HepVu-5 classes')

In [3]:
# loop through each state and load the shapefile and concatenate them into one dataframe
import geopandas as gpd
import pandas as pd

gdf = pd.DataFrame()
for state in states:
    print(state)
    try:
        gdf = pd.concat([gdf, gpd.read_file(f'/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/SVI2018 census tracts with death rate HepVu-5 classes/{state}/{state}.shp')])
    except:
        print(f'Error reading {state}')
        continue


/Users/h6x/anaconda3/envs/TDA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


VT
VA
SD
SC
UT
GA
MS
MT
MO
MA
AK
Error reading AK
KY
AL
NH
MN
MI
OK
IN
CO
CA
IA
CT
FL
WV
RI
WY
TX
PA
NC
ND
NM
NJ
ME
AR
NV
DC
Error reading DC
MD
KS
NE
HI
DE
AZ
NY
ID
OH
OR
IL
LA
WI
WA
TN


In [4]:
gdf.shape

(72491, 129)

In [5]:
gdf.head()

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Leng,Shape_Area,NOD_Rate,percentile,geometry
0,50,VERMONT,VT,50007,Chittenden,50007980000,"Census Tract 9800, Chittenden County, Vermont",1.141485,0,9,...,0,9,-999.0,-999.0,794,0.107289,0.000337,12.5,0,"POLYGON ((-73.16975 44.48154, -73.16620 44.484..."
1,50,VERMONT,VT,50001,Addison,50001960100,"Census Tract 9601, Addison County, Vermont",81.549039,3928,202,...,214,67,5.5,1.7,1619,0.743236,0.023958,12.9,0,"POLYGON ((-73.19680 44.26663, -73.16182 44.271..."
2,50,VERMONT,VT,50001,Addison,50001960200,"Census Tract 9602, Addison County, Vermont",47.443526,2736,23,...,151,75,5.5,2.7,1993,0.650662,0.017827,12.9,0,"POLYGON ((-73.39963 44.15533, -73.39873 44.162..."
3,50,VERMONT,VT,50001,Addison,50001960300,"Census Tract 9603, Addison County, Vermont",2.476118,2612,19,...,59,34,2.3,1.3,3360,0.109137,0.000744,12.9,0,"POLYGON ((-73.27275 44.17683, -73.24845 44.179..."
4,50,VERMONT,VT,50001,Addison,50001960400,"Census Tract 9604, Addison County, Vermont",124.237386,5083,179,...,183,41,3.6,0.8,2554,0.980535,0.040436,12.9,0,"POLYGON ((-73.43774 44.04501, -73.43199 44.063..."


In [6]:
# get the unque STCNTY values and their relevent Percentile column values
A = gdf[['STCNTY', 'percentile']].drop_duplicates().sort_values(by=['STCNTY'])


In [7]:
#make STCNTY column int
A['STCNTY'] = A['STCNTY'].astype(int)

In [8]:
A

,STCNTY,percentile
6,1001,2
12,1003,3
27,1005,0
29,1007,4
30,1009,4
...,...,...
70,56037,4
76,56039,0
78,56041,4
79,56043,4


In [9]:
# save A as a csv file
A.to_csv('/Users/h6x/ORNL/git/modeling-ideas/overdose modeling for entire country/data/processed data/svi with hepvu/2018/annotation 2018/annotation.csv', index=False)